In [1]:
#Fuel Cost Optimization Using Trust Region Method

In [2]:
import numpy as np;
import pandas as pd;

In [3]:
#Parameters used in solving the optimization problem

In [4]:
D = 10.0  #Distance of travel in the vehicle in kilometers
tau = 0.3  #Traffic index for medium traffic where tau belongs to {0.0,0.3,0.7} where 0.0 is free traffic, 0.3 is medium traffic and 0.7 is high traffic}
n = 2.0 #Number of passengers(driver + pillion)
w_avg = 70.0 #Average weight per person in kgs
C = 400 #Engine Power in cc
g = 0.01 #Road Gradient (fractional)
Pf = 102.0  #Fuel Price (Currency per litre)
Ct = 160.0  #Monetary cost of time (currency per hour)
s = 0.05  #Delay factor (hours per km at tau = 1)
v_min = 20.0   #Minimum speed of the vehicle
v_max = 120.0  #Maximum speed of the vehicle
v_max_eff = v_max

#Fuel Model Coefficients
c0 = 0.005
c1 = 2e-4
c2 = 1e-5
c3 = 0.001
c4 = 1e-5
c5 = 0.002

#Pre-calcualted Constants
B = Pf * D  # B = 100.0 * 10.0 = 1000.0
# K_new = c0 + c3*t + c4*n*w_avg + c5/C
K_new = c0 + c3*tau + c4*n*w_avg + c5/C # K_new = 0.0087

In [5]:
    #Function Definitions
    def mainfunction(v):
        fuel_cost = B*(K_new + c1*v + c2*(v**2))
        time_cost = Ct*((D/v) + s*tau*D)
        return fuel_cost + time_cost
    
    def f_prime(v):
        return B*(c1 + 2*c2*v) - Ct*((D/(v**2)))
    
    def f_double_prime(v):
        return B*2*c2 + 2*Ct*(D/(v**3))

    def proj(v):
        return np.clip(v, v_min, v_max)



In [6]:
def trust_region(v0):
    v_k = v0
    max_iterations = 100
    epsilon_g = 1e-6  #Stopping criteria : gradient norm threshold |f'(v)| <= epsilon_g

    #Trust region Parameters:
    Delta_k = 10.0  #Initial trust region radius (delta k)
    Delta_max = 50.0 #Maximum allowable radius
    eta = 1e-4

    print(f"Starting Trust-Region Optimization from v0 = {v0:.2f} km/h")

    for k in range(max_iterations):
        # 1. EVALUATE FUNCTIONS
        f_k = mainfunction(v_k)
        g_k = f_prime(v_k)
        H_k = f_double_prime(v_k) # Hessian calculation 

        # 2. Check Stopping Criterion
        if abs(g_k) <= epsilon_g:
            print(f"Convergence met in {k} iterations: |f'(v)| < {epsilon_g}")
            break

        # 3. SOLVE SUBPROBLEM
        # Unconstrained Newton step (p_star): minimizes the model without constraint
        p_star = -g_k / H_k
        
        # Constrained step (p_k): projection of p_star onto [-Delta_k, Delta_k]
        p_k = np.clip(p_star, -Delta_k, Delta_k)

        # 4. CALCULATE REDUCTIONS
        v_trial = v_k + p_k
        
        # Actual Reduction: f(v_k) - f(v_k + p_k)
        ActRed = f_k - mainfunction(v_trial)
        
        # Predicted Reduction: -(g_k*p_k + 0.5*H_k*p_k^2)
        PredRed = -(g_k * p_k + 0.5 * H_k * p_k**2)

        # 5. CALCULATE REDUCTION RATIO (rho_k)
        if PredRed <= 0:
            rho_k = -1.0 # Avoid division by zero/negative prediction; ensures rejection
        else:
            rho_k = ActRed / PredRed
        # 6. UPDATE POSITION (v) AND RADIUS (Delta)
        
        # Step Acceptance Check
        if rho_k > eta:
            # If model is good, accept the step and project onto feasible speed range
            v_k = proj(v_trial)
        # Else: v_k remains the same (step is rejected)
        
        # Radius Update Logic
        if rho_k < 0.25:
            Delta_k *= 0.25 # Shrink region: model was poor
        elif rho_k > 0.75 and np.abs(p_k) == Delta_k:
            Delta_k = min(2 * Delta_k, Delta_max) 
        # Else: Delta_k remains the same

        # Log progress (optional, but helpful)
        print(f"Iter {k+1}: v = {v_k:.4f}, f(v) = {mainfunction(v_k):.2f}, rho={rho_k:.3f}, Delta={Delta_k:.2f}")

    print("-" * 40)
    print(f"Final Optimal Speed (v*): {v_k:.4f} km/h")
    print(f"Minimum Total Cost: {mainfunction(v_k):.4f} currency")
    
    return v_k

if __name__ == '__main__':
    # Initial guess for speed
    trust_region(10.0)

Starting Trust-Region Optimization from v0 = 10.00 km/h
Iter 1: v = 20.0000, f(v) = 119.00, rho=1.324, Delta=10.00
Iter 2: v = 28.0590, f(v) = 101.62, rho=1.273, Delta=10.00
Iter 3: v = 35.6584, f(v) = 95.95, rho=1.187, Delta=10.00
Iter 4: v = 39.2516, f(v) = 95.32, rho=1.071, Delta=10.00
Iter 5: v = 39.7121, f(v) = 95.32, rho=1.008, Delta=10.00
Iter 6: v = 39.7180, f(v) = 95.32, rho=1.000, Delta=10.00
Convergence met in 6 iterations: |f'(v)| < 1e-06
----------------------------------------
Final Optimal Speed (v*): 39.7180 km/h
Minimum Total Cost: 95.3163 currency
